# Load Data

In [1]:
!pip install wordninja

import pandas as pd
import numpy as np
import wordninja
from nltk.tokenize import RegexpTokenizer

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Specify the path to our shared drive. Should work for all of us.
path = '/content/drive/Shareddrives/ANLY590_GROUP_4_PROJECT'

In [3]:
col_names_complete = ['track_id','race_date','race_number','program_number','trakus_index','latitude',
                      'longitude','distance_id','course_type','track_condition','run_up_distance','race_type',
                      'unknown', 'post_time', 'weight_carried','jockey','odds','position_at_finish']

col_names_start = ['track_id','race_date','race_number','program_number','weight_carried','jockey','odds','position_at_finish']

In [4]:
# Load the datasets.
complete = pd.read_csv(path + '/data/raw/nyra_2019_complete_raw.csv', names = col_names_complete, index_col=False)

start = pd.read_csv(path + '/data/raw/nyra_start_table_raw.csv', names = col_names_start, index_col=False)

race = pd.read_csv(path + '/data/raw/nyra_race_table_raw.csv', index_col=False)

tracking = pd.read_csv(path + '/data/raw/nyra_tracking_table_raw.csv', index_col=False)

pdf = pd.read_csv(path + '/data/raw/historical_results.csv', index_col=False)

aqu = pd.read_csv(path + '/data/raw/weather_data/aqu.csv')

bel = pd.read_csv(path + '/data/raw/weather_data/bel.csv')

sar = pd.read_csv(path + '/data/raw/weather_data/sar.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Codebook

#### nyra_start_table.csv

- track_id - 3 character id for the track the race took place at. AQU -Aqueduct, BEL - Belmont, SAR - Saratoga.
- race_date - date the race took place. YYYY-MM-DD.
- race_number - Number of the race. Passed as 3 characters but can be cast or converted to int for this data set.
- program_number - Program number of the horse in the race passed as 3 characters. Should remain 3 characters as it isn't limited to just numbers. Is essentially the unique identifier of the horse in the race.
- weight_carried - An integer of the weight carried by the horse in the race.
- jockey - Name of the jockey on the horse in the race. 50 character max.
- odds - Odds to win the race passed as an integer. Divide by 100 to derive the odds to 1. Example - 1280 would be 12.8-1.
- position_at_finish - An integer of the horse's finishing position. (added to the dataset 9/8/22)



#### nyra_race_table.csv

- track_id - 3 character id for the track the race took place at. AQU -Aqueduct, BEL - Belmont, SAR - Saratoga.
- race_date - date the race took place. YYYY-MM-DD.
- race_number - Number of the race. Passed as 3 characters but can be cast or converted to int for this data set.
- distance_id - Distance of the race in furlongs passed as an integer. Example - 600 would be 6 furlongs.
- course_type - The course the race was run over passed as one character. M - Hurdle, D - Dirt, O - Outer turf, I - Inner turf, T - turf.
- track_condition - The condition of the course the race was run on passed as three characters. YL - Yielding, FM - Firm, SY - Sloppy, GD - Good, FT - Fast, MY - Muddy, SF - Soft.
- run_up_distance - Distance in feet of the gate to the start of the race passed as an integer.
- race_type - The classification of the race passed as as five characters. STK - Stakes, WCL - Waiver Claiming, WMC - Waiver Maiden Claiming, SST - Starter Stakes, SHP - Starter Handicap, CLM - Claiming, STR - Starter Allowance, AOC - Allowance Optionl Claimer, SOC - Starter Optional Claimer, MCL - Maiden Claiming, ALW - Allowance, MSW - Maiden Special Weight.
- purse - Purse in US dollars of the race passed as an money with two decimal places.
- post_time - Time of day the race began passed as 5 character. Example - 01220 would be 12:20.



#### nyra_tracking_table.csv

- track_id - 3 character id for the track the race took place at. AQU -Aqueduct, BEL - Belmont, SAR - Saratoga.
- race_date - date the race took place. YYYY-MM-DD.
- race_number - Number of the race. Passed as 3 characters but can be cast or converted to int for this data set.
- program_number - Program number of the horse in the race passed as 3 characters. Should remain 3 characters as it isn't limited to just numbers. Is essentially the unique identifier of the horse in the race.
- trakus_index - The common collection of point of the lat / long of the horse in the race passed as an integer. From what we can tell, it's collected every 0.25 seconds.
- latitude - The latitude of the horse in the race passed as a float.
- longitude - The longitude of the horse in the race passed as a float.



#### nyra_2019_complete.csv 
This file is the combined 3 files into one table. The keys to join them trakus with race - track_id, race_date, race_number. To join trakus with start - track_id, race_date, race_number, program_number.

- track_id - char(3)
- race_date - date
- race_number - char(3)
- program_number - char(3)
- trakus_index - int
- latitude - float
- longitude - float
- distance_id - int
- course_type - char(1)
- track_condition - char(3)
- run_up_distance - int
- race_type - char(5)
- post_time - char(5)
- weight_carried - int
- jockey - char(50)
- odds - int
- position_at_finish - An integer of the horse's finishing position. (added to the dataset 9/8/22)

In [5]:
complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5228430 entries, 0 to 5228429
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   track_id            object 
 1   race_date           object 
 2   race_number         int64  
 3   program_number      object 
 4   trakus_index        int64  
 5   latitude            float64
 6   longitude           float64
 7   distance_id         int64  
 8   course_type         object 
 9   track_condition     object 
 10  run_up_distance     int64  
 11  race_type           object 
 12  unknown             float64
 13  post_time           int64  
 14  weight_carried      int64  
 15  jockey              object 
 16  odds                int64  
 17  position_at_finish  int64  
dtypes: float64(3), int64(8), object(7)
memory usage: 718.0+ MB


In [6]:
start.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14915 entries, 0 to 14914
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   track_id            14915 non-null  object
 1   race_date           14915 non-null  object
 2   race_number         14915 non-null  int64 
 3   program_number      14915 non-null  object
 4   weight_carried      14915 non-null  int64 
 5   jockey              14915 non-null  object
 6   odds                14915 non-null  int64 
 7   position_at_finish  14915 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 932.3+ KB


In [7]:
race.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   track_id         2000 non-null   object
 1   race_date        2000 non-null   object
 2   race_number      2000 non-null   int64 
 3   distance_id      2000 non-null   int64 
 4   course_type      2000 non-null   object
 5   track_condition  2000 non-null   object
 6   run_up_distance  2000 non-null   int64 
 7   race_type        2000 non-null   object
 8   purse            2000 non-null   int64 
 9   post_time        2000 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 156.4+ KB


In [8]:
tracking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5228430 entries, 0 to 5228429
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   track_id        object 
 1   race_date       object 
 2   race_number     int64  
 3   program_number  object 
 4   trakus_index    int64  
 5   latitude        float64
 6   longitude       float64
dtypes: float64(2), int64(2), object(3)
memory usage: 279.2+ MB


In [9]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14354 entries, 0 to 14353
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Track              14354 non-null  object
 1   Date               14354 non-null  object
 2   Race               14354 non-null  int64 
 3   LastRaced          14354 non-null  object
 4   Pgm                14354 non-null  object
 5   HorseName(Jockey)  14354 non-null  object
 6   Wgt                14354 non-null  object
 7   M/E                14354 non-null  object
 8   PP                 14354 non-null  int64 
 9   Start              14354 non-null  object
 10  0.25               14354 non-null  object
 11  0.5                14354 non-null  object
 12  0.75               14354 non-null  object
 13  Str                14354 non-null  object
 14  Fin                14354 non-null  object
 15  Odds               14354 non-null  object
 16  Comments           14354 non-null  objec

# Merging

### Our questions:

(1) Can we accurately predict a horse’s finish position and what factors are most influential in determining this? 

(2) What factors are most deterministic in predicting the odds of a horse to win a race? 

(3) What factors are most influential in predicting the amount of money put into a race? 

(4) What factors determine the amount of time it takes a horse to finish a race? 

(5) Does the track type, course type, race type, or track condition play a role in determining horse performance or health? 

(6) What role does the geography and position of the horse in a race play in its outcome? 


Since the complete dataframe was created by merging all three datasets including the real-time position (traking) dataframe, I don't think we need to use this complete dataframe. I think the merged start and race should be enough for answering the first five question. We can use the tracking df to approach the last question. 

In [10]:
# Change the type for some columns.
race['distance_id'] = race['distance_id'].astype('int64')
race['run_up_distance'] = race['run_up_distance'].astype('int64')
race['purse'] = race['purse'].astype('int64')
start['program_number'] = start['program_number'].apply(lambda x : x.replace(" ", ""))

In [11]:
# Locate the right place to separate the column.
def find_numeric(string):
  place = False
  for i in range(0, len(string)):
    if not string[i].isalpha():
      continue
    else:
      return i + 5

In [12]:
# Clening up the scraped pdf csv.
pdf['LastRaced'] = pdf['LastRaced'].replace('---', np.nan, regex=True)
pdf['Track'] = pdf['Track'].apply(lambda x : x[0:3])
pdf['last_race_track'] = pdf['LastRaced'].apply(lambda x : x[-4:-1] if x is not np.nan else np.nan)
pdf['last_race_date'] = pd.to_datetime(pdf['LastRaced'].apply(lambda x : x[0 : find_numeric(x)] if x is not np.nan else np.nan), 
                                       format="%d%b%y", 
                                       errors='coerce')
pdf['Date'] = pd.to_datetime(pdf['Date'], format="%B%d,%Y")
pdf['day_since_last_race'] = pdf['Date'] - pdf['last_race_date']

In [13]:
# Splt the commnets to tokens,
res = []
tokenizer = RegexpTokenizer(r'\w+')
for i in pdf.Comments:
  if type(i) == str:
    phrase_list = tokenizer.tokenize(i)
    splited_tokens = [wordninja.split(i) for i in phrase_list]
    flat_splited_tokens = [j for sublist in splited_tokens for j in sublist]
    res.append(flat_splited_tokens)    
  else:

    res.append(np.nan)

In [14]:
pdf['Comments'] = res

Here I added two 'last race' column. One depicts the location of last race of this horse. The other one is added by calculating the days between the current race and last race. 

In [15]:
# Subset and rename,
historical_results = pdf[['Track','Date','Race','Pgm','last_race_track','day_since_last_race','Comments']].rename(
    columns = {'Track':'track_id',	'Date':'race_date',	'Race':'race_number',	'Pgm':'program_number'})
historical_results.race_date = historical_results.race_date.astype('str')

In [16]:
# Merge the dfs.
start_race = start.merge(
    race, how = 'left', on=['track_id','race_date','race_number']).merge(
        historical_results, how = 'left', on=['track_id','race_date','race_number','program_number'])

In [17]:
# Merge the weather data.
dfmerged = pd.concat([aqu, bel, sar], axis=0, ignore_index=True)
dfmerged = dfmerged.drop(dfmerged.columns[[0]],axis = 1)

In [18]:
final = pd.merge(start_race, dfmerged,  how='left', left_on=["track_id","race_date"], right_on = ['local','day']).drop(
    ['local', 'day'], axis = 1)

In [19]:
# final.to_csv(path + "/data/merged_data/joined_data_without_position.csv", encoding='utf-8')

The RNN model can only be trained on a subset of the tracking dataframe since it has the positions of all horses and races. So I think we can frame this part like "we are able to build predictive model for positions in a race for any horses so that the stakeholders can be better informed during a race to make decisions".

In [20]:
complete.head()

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude,distance_id,course_type,track_condition,run_up_distance,race_type,unknown,post_time,weight_carried,jockey,odds,position_at_finish
0,AQU,2019-01-01,9,6,72,40.672902,-73.827607,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
1,AQU,2019-01-01,9,6,73,40.672946,-73.827587,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
2,AQU,2019-01-01,9,6,74,40.672990,-73.827568,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
3,AQU,2019-01-01,9,6,63,40.672510,-73.827781,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
4,AQU,2019-01-01,9,6,64,40.672553,-73.827762,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
